In [1]:
import operator
import re

from pyspark import SparkConf, SparkContext

In [2]:
sc = SparkContext(conf=SparkConf().setAppName("FindPath").setMaster("local"))

In [3]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

def step(item):
    prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
    return (next_v, prev_d + 1)

def complete(item):
    v, old_d, new_d = item[0], item[1][0], item[1][1]
    return (v, old_d if old_d is not None else new_d)

n = 400  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

In [ ]:
x = 12
y = 34
d = 0

distances = sc.parallelize([(x, d)]).partitionBy(n)

found = False

while True:
    candidates = distances.join(forward_edges, n).map(step)
    new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist()
    count = new_distances.filter(lambda i: i[1] == d + 1).count()

    if candidates.filter(lambda pair: pair[0] == y).count():
        found = True
        break
    if found or count > 0:
        d += 1
        distances = new_distances
    else:
        break


In [ ]:
path = [y]

step = d
next_ = y

while found and step:
    step_candidats = candidates.filter(lambda pair: pair[1] == step).keys().map(lambda item: (item, next_))
    path_candidates = forward_edges.intersection(step_candidats)
    # get any candidat
    next_ = path_candidates.first()[0]
    # decrease step and found next
    step -= 1
    path.append(next_)

path.append(x)

In [ ]:
path.reverse()
print ','.join(map(str, path))